# 查看活动和交互

在第一个笔记本 `Personalize_BuildCampaign.ipynb` 中，您使用 Amazon Personalize 的深度学习成功构建和部署了推荐模型。

本笔记本将进行详述，并将指导您添加对用户实时行为作出反应的功能。如果他们在浏览电影时意图发生变化，您将看到基于该行为的经修订的推荐。

它还将展示演示代码，用于模拟用户在返回推荐之前选择电影的行为。

接下来，我们首先导入需要与 Personalize 交互的库

In [ ]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import uuid

以下行将从第一个笔记本中检索您的共享变量。

In [ ]:
%store -r

In [ ]:
# Setup and Config
# Recommendations from Event data
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's Event Streaming
personalize_events = boto3.client(service_name='personalize-events')

## 创建事件跟踪器

在您的推荐系统能够响应实时事件之前，您将需要一个事件跟踪器，以下代码将生成一个事件跟踪器，并可用于此实验室。您可以随意为之采用更巧妙的名称。

In [ ]:
response = personalize.create_event_tracker(
    name='MovieClickTracker',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']

In [ ]:
event_tracker_arn = response['eventTrackerArn']

## 配置源数据

在上面，您将看到您的追踪 ID，并且此 ID 已分配给变量，因此您无需采取进一步措施。下面的代码行将设置用于推荐的数据，以便您以后呈现电影列表。

In [ ]:
# First load items into memory
items = pd.read_csv('./ml-100k/u.item', sep='|', usecols=[0,1], encoding='latin-1', names=['ITEM_ID', 'TITLE'], index_col='ITEM_ID')

def get_movie_title(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)-1
    return items.loc[movie_id]['TITLE']

## 获得推荐

首先，我们将再次从之前的笔记本中呈现推荐：

In [ ]:
recommendations_df

## 模拟用户行为

下面的代码行提供了模拟用户与特定项目交互的代码示例，然后您将获得不同于开始时的推荐。

In [ ]:
session_dict = {}

In [ ]:
def send_movie_click(USER_ID, ITEM_ID):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[USER_ID]
    except:
        session_dict[USER_ID] = str(uuid.uuid1())
        session_ID = session_dict[USER_ID]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Make Call
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= USER_ID,
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': 'EVENT_TYPE',
        'properties': event_json
        }]
)

此代码行的正下方将更新跟踪器，就像用户单击了特定标题一样。


如果下面的单元格生成的表不会转变推荐，则只需在上面的单元格中尝试另一个随机的 3 位数字，然后再次运行这两个单元格。您将看到由推荐生成的第三列。

In [ ]:
# Pick a movie, we will use ID 270 or Gattaca
movie_to_click = 270
movie_title_clicked = get_movie_title(movie_to_click)
send_movie_click(USER_ID=str(user_id), ITEM_ID=movie_to_click)

执行此阻止命令后，您将在推荐中看到更改，因为您已启用事件跟踪功能，并且已将事件发送到服务。

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(user_id),
)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']

recommendation_list = []

for item in item_list:
    title = get_movie_title(item['itemId'])
    recommendation_list.append(title)
    
new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_title_clicked])

recommendations_df = recommendations_df.join(new_rec_DF)
recommendations_df

## 结论

现在，您可以看到，通过更改用户与之交互的电影来更改推荐，此系统可修改为用户与项目集合交互的任何应用程序。这些工具随时可用，可用于下拉并开始浏览您现有数据可以实现的结果。

执行以下单元格以存储清理笔记本所需的值。

最后，当您准备好从账户中删除项目时，打开 `Cleanup.ipynb` 笔记本并执行步骤。


In [ ]:
%store event_tracker_arn